In [1]:
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [3]:
llm.invoke("What is the capital of France?").content

'The capital of France is Paris.'

In [5]:
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(
        None, description="Query that is optimized to search the web."
    )
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )

structured_llm = llm.with_structured_output(SearchQuery)
output = structured_llm.invoke(
    "How does Calcium CT score relate to high cholesterol?"
)
output.search_query, output.justification

('Calcium CT score high cholesterol relationship',
 'The user is asking about the relationship between Calcium CT score and high cholesterol, which requires a web search to provide an accurate answer about medical correlations and risk factors. ')

In [15]:
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What is 2 times 3?")

msg.tool_calls[0]["name"], msg.tool_calls[0]["args"]

('multiply', {'a': 2.0, 'b': 3.0})

In [19]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

def generate_joke(state: State):
    """First LLM call to generate initial joke"""
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

def check_pounchline(state: State):
    """Gate function to check if the joke has a punchline"""
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Pass"
    return "Fail"

def improve_joke(state: State):
    """Second LLM call to improve the joke"""
    msg = llm.invoke(
        f"Make this joke funnier by adding wordplay: {state['joke']}"
    )
    return {"improved_joke": msg.content}

def polish_joke(state: State):
    """Third LLM call to final Polish the joke"""
    msg = llm.invoke(
        f"Add a surprising twist to this joke: {state['improved_joke']}"
    )
    return {"final_joke": msg.content}

In [ ]:
workflow = StateGraph(State)
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)